<a href="https://colab.research.google.com/github/HisameOgasahara/HF_transformers/blob/main/HFHT_4_2_%EB%AA%A8%EB%8D%B8_%EC%B4%88%EA%B8%B0%ED%99%94_%ED%9B%84_%ED%95%99%EC%8A%B5%2C_%ED%8F%89%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사전준비

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install datasets==2.20.0 transformers accelerate -U  evaluate==0.4.0 scikit-learn==1.4.2  #뒤에 두개는 평가용

# 4.1.2 모델 초기화 후 학습: mycfg를 받아서 모델 초기화

#### from_pretrained로 모델을 불러오는게 아니다! 다시말해 이 노트북(아래 cell 제외)에선 웨이트는 학습된 걸 가져오는 게 아니라 랜덤으로 된 상태, 즉 바닥부터 학습을 시작하는 것이다.

In [3]:
from datasets import load_dataset
from transformers import BertTokenizerFast #BertTokenizerFast는 BERT 모델에 사용되는 토크나이저의 빠른 버전입니다. (Rust 기반으로 구현되어 속도가 빠릅니다.)

dataset = load_dataset("klue", "ynat")
model_name = "/content/drive/MyDrive/HFbookPractice/trained_model/25_02_19/MyTokenizer" #원래는 이전 노트북에서 학습한 토크나이저를 가져와야 하는데..
tokenizer = BertTokenizerFast.from_pretrained(model_name) #이건 토크나이저지 웨이트가 아니다!

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
from transformers import BertConfig #BertConfig는 BERT 모델의 설정을 정의하는 클래스입니다.
cfg = BertConfig #BertConfig 클래스를 cfg 변수에 할당합니다 (클래스 자체를 할당)
print(cfg) # 클래스 정의를 보여주는 것이며, 실제 설정 값을 보여주는 것은 아닙니다. 기본 설정 값을 보려면 print(BertConfig()) 처럼 인스턴스를 생성해서 출력해야 합니다.
print(BertConfig())

<class 'transformers.models.bert.configuration_bert.BertConfig'>
BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.49.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [5]:
mycfg = BertConfig(vocab_size=tokenizer.vocab_size)
#BertConfig를 사용하여 새로운 BERT 모델 설정을 생성합니다. vocab_size를 이전에 로드한 토크나이저의 어휘 크기(tokenizer.vocab_size)로 설정합니다.
#이렇게 하면 모델의 어휘 크기가 토크나이저와 일치하게 됩니다. 다른 파라미터들(hidden size, number of layers 등)은 BertConfig의 기본값을 사용합니다.

In [6]:
from transformers import BertForMaskedLM

model = BertForMaskedLM(mycfg) #BertForMaskedLM 클래스를 사용하여 모델을 초기화합니다. 이때 앞에서 정의한 사용자 정의 설정(mycfg)을 사용합니다.
print(model.config) # 초기화된 모델의 설정을 출력합니다. vocab_size가 tokenizer.vocab_size와 일치하고, 다른 설정들은 BertConfig의 기본값을 사용

BertConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.49.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 24000
}



In [7]:
# 설정을 저장할 디렉토리 경로 지정
config_output_dir = "./my_bert_config"

# 모델 설정을 config_output_dir에 저장
model.config.save_pretrained(config_output_dir)

### 학습 하이퍼파라미터 설정

In [11]:
!pip install --upgrade transformers #4.41.2 버젼을 하니까 문제가 생겨서 업그레이드

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 59.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [9]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

datasets = dataset.map(
    lambda x: tokenizer(x["title"]),
    batched=True,
    batch_size=1000,
    remove_columns=dataset.column_names["train"],
)

args = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    max_steps=5, #원래 코드는 1000. CPU로 동작 확인만 할거라.
    evaluation_strategy="steps", #eval_strategy="steps"  추후 이걸로 변경된다고
    logging_strategy="steps",
    logging_steps=1,
    logging_dir="/content/drive/MyDrive/HFbookPractice/trained_model/25_02_19/a/logs",
    output_dir="/content/drive/MyDrive/HFbookPractice/trained_model/25_02_19/a/ckpt",
    report_to="tensorboard",
)

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["validation"].select(range(30)),#원래는 datasets["validation"]인거 cpu쓰니 30개만 하는 걸로.
    tokenizer=tokenizer, #Trainer는 이제 이미 전처리된 데이터셋을 받아서 학습을 진행하니 추후엔 이거 없애라고 함.
    data_collator=collator,
)

trainer.train()
trainer.save_model("drive/MyDrive/HFbookPractice/trained_model/25_02_19/a/MyBertModel")

Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

<ipython-input-9-30b6d92bc817>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
1,8.072000,10.190947
2,10.387900,10.094158
3,10.230300,10.173585
4,9.971200,10.140940
5,10.155500,10.059534


# 평가

# 4.2.1 evaluate 라이브러리 사용: 평가하는 2가지 방법

#### 위에서 훈련한 걸 이걸로 평가 X. 아래 훈련에 이용될 거.

### 공통

In [10]:
import evaluate #HF에서 다양한 평가 지표들을 미리 구현해 놓았고, 사용자가 쉽게 불러와 사용할 수 있도록 인터페이스를 제공

acc = evaluate.load("accuracy")
#evaluate.load() 함수는 라이브러리에 내장된 평가 지표를 불러오는 함수입니다. "accuracy"는 정확도 지표를 나타내는 문자열입니다.
#이 함수는 정확도를 계산하는 데 필요한 도구를 acc 변수에 담아줍니다.

In [11]:
metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"]) #여러 개의 평가 지표를 한 번에 계산하기 위해 evaluate.combine() 함수를 사용
metrics.compute(predictions=[1,0,0,1], references=[0,1,0,1]) #위에서 학습한 모델 예측이 아닌 임의로 [1,0,0,1]는 예측과 실제 정답(ground truth) [0,1,0,1]값들의 리스트를 제공
#redictions와 references는 같은 길이를 가져야 하며, 각 위치에 대응되는 예측값과 실제 정답을 가져야 합니다.

{'accuracy': 0.5, 'f1': 0.5, 'precision': 0.5, 'recall': 0.5}

In [12]:
print(metrics)

### 방법1

In [16]:
for y, pred in zip([0,1,0,1], [1,0,0,1]): #실제 정답 리스트 [0,1,0,1]과 예측값 리스트[1,0,0,1] 을 묶어서 순회할 수 있도록 합니다. zip 함수는 각 리스트의 동일한 인덱스에 있는 요소들을 튜플로 묶어줍니다.
  metrics.add(predictions=pred, references=y) #metrics 객체에 개별 예측값과 실제 정답을 하나씩 추가하는 함수입니다.

metrics.compute()

#이 줄은 루프가 반복될 때마다 하나의 예측-정답 쌍을 metrics 객체에 누적합니다.(이는 추후 훈련시의 accuracy 계산에 이용될거라)

{'accuracy': 0.5, 'f1': 0.5, 'precision': 0.5, 'recall': 0.5}

### 방법2: 배치 단위 => 이 부분은 제대로 확인해 볼 것. gemini 출력이 이상한 거 같음.

In [17]:
for y, preds in zip([[0,1],[0,1]], [[1,0],[0,1]]): # 실제 정답 배치 리스트 [[1,0],[1,0]와 예측값 배치 리스트 [[0,1],[0,1]]를 묶어서 순회합니다. 4개 샘플, 총 8개 샘플
                                                   # [[0,1],[0,1]]은 실제 정답(references) 배치를, [[1,0],[0,1]] 는 예측값(predictions) 배치
                                                   # 4개의 각 내부 리스트 ([0,1], [0,1], [1,0], [0,1])는 하나의 배치
                                                   # 외부 리스트 ([[0,1],[0,1]], [[1,0],[0,1]]) 는 여러 배치들을 담고 있는 리스트입니다. (이 예시에서는 각각 2개의 배치)
  metrics.add_batch(predictions=preds, references=y) #metrics 객체에 예측값 배치와 실제 정답 배치를 한 번에 추가

metrics.compute()

{'accuracy': 0.5, 'f1': 0.5, 'precision': 0.5, 'recall': 0.5}

#####  metrics.add()는 개별 값 (숫자)를 입력으로 받는 반면, metrics.add_batch()는 값들의 리스트 (배치)를 입력으로 받습니다.

##### metrics.compute() 메서드는 add_batch() 및 이전의 add() 호출을 통해 누적된 모든 예측과 정답을 기반으로 평가 지표를 계산하고 결과를 반환합

##### 중요: 두 번째 metrics.compute()는 코드 블록 1에서 add()로 추가된 데이터와 코드 블록 2에서 add_batch()로 추가된 데이터를 모두 포함하여 계산합니다. 즉, 평가는 누적 방식으로 진행됩니다.

# 4.2.2 커스텀 매트릭스 만들기

In [18]:
def simple_accuracy(preds, labels):
    return {"accuracy": (preds == labels).to(float).mean().item()} #모델의 예측값(preds)과 실제 정답 레이블(labels)을 입력으로 받아 정확도(accuracy) 를 계산하고, 결과를 딕셔너리 형태로 반환합니다.

# 4.2.3 평가 함수를 설정해 Trainer 적용해 써보자

In [1]:
import evaluate

def custom_metrics(pred):
    f1 = evaluate.load("f1")
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    return f1.compute(predictions=preds, references=labels, average="micro")

In [2]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    default_data_collator
)

model_name = "bert-base-multilingual-cased" #가벼운걸로 변경
#model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7)

dataset = load_dataset("klue", "ynat")

def tokenize_function(sample):
    result = tokenizer(
        sample["title"],
        padding="max_length",
    )
    return result

datasets = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=1000,
    remove_columns=["guid", "title", "url", "date"]
)
print(datasets)

args = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    max_steps=2,#원래는 500
    eval_strategy="steps",
    logging_strategy="steps",
    logging_steps=2, #원래는 50
    logging_dir="drive/MyDrive/HFbookPractice/trained_model/25_02_19/b/logs",
    save_strategy="steps",
    save_steps=5, #원래는 50
    output_dir="drive/MyDrive/HFbookPractice/trained_model/25_02_19/b/ckpt",
    report_to="tensorboard",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["validation"].select(range(2)),#원래는 datasets["validation"]인거 30개만 하는 걸로.
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    compute_metrics=custom_metrics, #여기에 위에서 설정한 평가 메트릭을 추가
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9107
    })
})


<ipython-input-2-87c53c27ca76>:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [3]:
%load_ext tensorboard
%tensorboard --logdir /content/logs

Reusing TensorBoard on port 6006 (pid 14397), started 0:07:57 ago. (Use '!kill 14397' to kill it.)

<IPython.core.display.Javascript object>

In [ ]:
trainer.train() #계속 ram 초과해서 세션 다운된다. 나중에 GPU로 해볼 것.